# Construyendo un Motor de Búsqueda desde 0

Pasos:
1. Configurar el ambiente para el desarrollo.
   - Configurar ambiente virtual Python.
   - Configurar Elasticsearch instance.
2. Normalizar contenido para búsqueda. Usaremos una versión reducida de este conjunto de datos disponible públicamente: https://www.kaggle.com/dhruvildave/wikibooks-dataset
3. Crear indice Elasticsearch con contenido normalizado.
4. Construir sentencias de búsqueda para Elasticsearch.
5. Medir el desempeño de mi motor con cada sentencia de búsqueda y escoger la mejor.
6. Crear UI y ensamblar todos lo componentes.
7. BONUS: Integrar modelos de lenguaje para:
    - Mejorar reranking
    - Extraer y mostrar respuesta al usuario contenida en los parrafos

### 1. Configurar el ambiente para el desarrollo

In [ ]:
# instalar dependencias requeridas
!pip install -r requirements.txt

In [ ]:
# iniciar elasticsearch instance
!docker-compose up -d

### 2. Normalizar contenido para búsqueda

In [ ]:
# lectura del dataset archivo zip e imprimir numero total de registros
import pandas as pd
dataset = pd.read_csv("dataset/reduced-es-books-dataset.csv.zip")
dataset = dataset[['title', 'url', 'body_html']]
print(f"Total libros: {len(dataset)}")

#### 2.1 Reconocimiento del formato

In [ ]:
# convertir dataset a listado de dicts e imprimir alguno de los elementos
books = dataset.to_dict(orient="records")

In [ ]:
print([b['title'] for b in books[:2]])

In [ ]:
from IPython.core.display import display, HTML
def display_html(input_html):
    display(HTML(input_html))

In [ ]:
display_html(books[0]['body_html'])

#### 2.2 Conversión HTML a formato intermedio Markdown

In [ ]:
# HTML de referencia
html = "<b>Titulo de documento</b>" + \
"<br /><br /><br />" + \
"Rutas" + \
"<ul>" + \
"<li><em>Ruta</em> <b>1</b></li>" + \
"<li><a href='#example'>Ruta 2</a></li>" + \
"</ul>" + \
"<a href='https://www.misiontic2022.gov.co/portal/Secciones/Inscripciones-2021/'>Requisitos Ruta 2</a>:" + \
"<p>Si eres mayor de 15        años, quieres aprender a programar y tienes la disponibilidad de 30 horas semanales para realizar la formación. Deberás registrate en la web, seleccionar la ruta 2 y diligenciar el formulario de inscripción.</p>" + \
"Universidades:" + \
"<br />" + \
"* UniNorte<br />· UniSimon<br />- Universidad Nacional<br />La universidad nacional ...<br />- UNAB" + \
"<br /><br /><br />" + \
"<span>[</span><a href='#example' title='Editar sección'>editar</a><span>]</span>Tiempos:" + \
"<br />" + \
"<table><tr><th>Ruta</th><th>Tiempo</th><th>Edad</th></tr><tr><td>Ruta 1</td><td>11 horas</td><td>12</td></tr><tr><td>Ruta 2</td><td>30 horas</td><td>18</td></tr></table>" + \
"<img src='https://blog-assets.lightspeedhq.com/img/2020/04/bbcdda07-xxxxxgoogle-shopping.jpg' alt='mintic 2022' width='500'>"
display_html(html)

In [ ]:
from html2text import HTML2Text
parser = HTML2Text()
md = parser.handle(html)
print(md)

#### 2.3 Ajustar conversor para lectura óptima

In [ ]:
# ignorar enlaces, ignorar enfasis, ignorar imagenes y no ajustar ancho de texto
from html2text import HTML2Text
def convert_from_html_to_md(input_html):
    parser = HTML2Text()
    parser.ignore_links = True
    parser.ignore_emphasis = True
    parser.ignore_images = True
    parser.body_width = 0
    md = parser.handle(input_html)
    return md

In [ ]:
md = convert_from_html_to_md(html)
print(md)

#### 2.4 Normalizar listas

In [ ]:
# reescribir los elementos de lista usando un correcto formato de Markdown (eg.  * list item) - re.sub
import re
def normalise_md_lists(input_md):
    return re.sub(r'(^|\n)\\?[·*-]( \w)', r'\1  *\2', input_md)

In [ ]:
normalised_lists_md = normalise_md_lists(md)
print(normalised_lists_md)

#### 2.5 Normalizar tablas

In [ ]:
# 1. "| " -> " | "
def normalise_md_table(input_md):
    normalised = input_md.replace("| ", " | ")
    # 2. eliminar "-----|----|---"
    normalised = re.sub(r'\-+\|.*\n', '\n', normalised)
    # 3. "    | " -> " | "
    normalised = re.sub(r' +\| +', ' | ', normalised)
    return normalised

In [ ]:
normalised_table_md = normalise_md_table(normalised_lists_md)
print(normalised_table_md)

#### 2.6 De vuelta a HTML simplificado (MD -> HTML)

In [ ]:
from mistletoe import markdown
def convert_md_to_html(input_md):
    return markdown(input_md)

In [ ]:
simplified_html = convert_md_to_html(normalised_table_md)

In [ ]:
display_html(simplified_html)

#### 2.7 Conversión HTML a Text

In [ ]:
from bs4 import BeautifulSoup
def convert_html_to_text(input_html):
    return BeautifulSoup(input_html).getText()

In [ ]:
text = convert_html_to_text(simplified_html)
print(text)

#### 2.8 Manejo extra espacios

In [ ]:
def clean_up_spaces(input_text):
    # 1. eliminar espacios a los extremos
    cleaned = input_text.strip()
    # 2. reemplazar tabulaciones y espacios dobles por uno dentro del texto
    cleaned = re.sub(r'[\t ]+', ' ', cleaned)
    # 3. reemplazar salto de linea dobles por uno dentro del texto
    cleaned = re.sub(r'\n+', '\n', cleaned)
    return cleaned

In [ ]:
no_extra_spaces_text = clean_up_spaces(text)
print(no_extra_spaces_text)

#### 2.9 Borrar artefactos innecesarios

In [ ]:
# borrar enlaces [editar]
def remove_edit_link(input_text):
    return input_text.replace("[editar]", "")

In [ ]:
no_edit_links_text = remove_edit_link(no_extra_spaces_text)
print(no_edit_links_text)

#### 2.10 Divide el texto en párrafos

In [ ]:
def extract_paragraphs(input_text):
    return input_text.split("\n")

In [ ]:
paragraphs = extract_paragraphs(no_edit_links_text)
paragraphs

#### 2.11 Filtrar párrafos - definir datos en alcance

In [ ]:
# Alcance definido:
# 1. solo incluye los párrafos con más de 50 letras
def filter_paragraphs(paragraph_list):
    filtered_list = [par for par in paragraph_list if len(par) > 50]
    return filtered_list

In [ ]:
filtered_paragraphs = filter_paragraphs(paragraphs)
print(filtered_paragraphs)

#### 2.12 Crear función global de normalización de contenido (unir piezas)

In [ ]:
def normalise_content(input_html):
    md = convert_from_html_to_md(input_html)
    normalised_lists_md = normalise_md_lists(md)
    normalised_table_md = normalise_md_table(normalised_lists_md)
    simplified_html = convert_md_to_html(normalised_table_md)
    text = convert_html_to_text(simplified_html)
    no_extra_spaces_text = clean_up_spaces(text)
    no_edit_links_text = remove_edit_link(no_extra_spaces_text)
    paragraphs = extract_paragraphs(no_edit_links_text)
    filtered_paragraphs = filter_paragraphs(paragraphs)
    return filtered_paragraphs

#### 2.13 Crear función de normalización de títulos

In [ ]:
# Wikilibros: Wikichicos/La Tierra/Los continentes/Europa/Clima -> La Tierra, Los continentes, Europa, Clima
import re
def normalise_title(input_title):
    clean_title = re.sub(r'Wikilibros|Wikichicos', "", input_title)
    clean_title = clean_title.strip(":/ ")
    clean_title = ", ".join(clean_title.split("/"))
    return clean_title
title = "Wikilibros: Wikichicos/La Tierra/Los continentes/Europa/Clima"
print(normalise_title(title))

#### 2.14 Crear elementos que indexaremos a Elasticsearch - formato definido:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": ""
}

In [ ]:
from tqdm import tqdm
normalized_content = list()
for book_id, book in tqdm(enumerate(books)):
    paragraphs = normalise_content(book['body_html'])
    title = normalise_title(book['title'])
    for par_id, par_text in enumerate(paragraphs):
        par = {"bookId": book_id, 
               "bookTitle": title, 
               "bookURL": book['url'], 
               "paragraphId": f"{book_id}-{par_id}",
               "paragraphText": par_text}
        normalized_content.append(par)

In [ ]:
# imprimir total numero de parrafos a almacenar en ES
print(f"Total parrafos: {len(normalized_content)}")

In [ ]:
# persistimos contenido normalizado en forma de parrafos al disco duro en formato JSON en dataset/processed/ (crear directorio si no existe)
import json
import os
os.mkdir("dataset/processed")
with open("dataset/processed/processed.json", "w") as f:
    json.dump(normalized_content, f, indent=4)

### 3. Crear indice en Elasticsearch con contenido normalizado

##### 3.1. Instanciamos cliente para poder interactuar con Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

##### 3.2. Función para crear índice (indexar contenido) en Elasticsearch

In [ ]:
import os
from elasticsearch import helpers

def create_index(index_action_list, index_name):
    if es.indices.exists(index=index_name):
        print("deleting existing elasticsearch index...")
        es.indices.delete(index=index_name, ignore=[400, 404])
    
    print("creating elasticsearch index...")
    request_body = {
        "mappings": {
            "properties": {
                "bookTitle": {
                    "type": "text",
                    "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                    },
                    "analyzer": "spanish"
                },
                "paragraphText": {
                    "type": "text",
                    "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                    },
                    "analyzer": "spanish"
                }
            }
        }
    }
    es.indices.create(index=index_name, body=request_body)

    if index_action_list:
        print("indexing to elasticsearch...")
        for success, info in helpers.parallel_bulk(es, index_action_list, thread_count=os.cpu_count()):
            if not success:
                print('A document failed:', info)
        print("done indexing")
    else:
        print("no items to index")

##### 3.4. Creación del índice

In [ ]:
# lectura de archivo con parrafos procesados
import json
with open("dataset/processed/processed.json") as fp:
    paragraphs = json.load(fp)

In [ ]:
# creación de indice con contenido normalizado generado en el paso 2
index_name = "wikibooks-search-index"
action_list = list()
for par in normalized_content:
    action_list.append({
        "_index": index_name,
        "_id": par['paragraphId'],
        "_source": par
    })
create_index(action_list, index_name)

### 4. Construir sentencias de búsqueda (query) para Elasticsearch

#### 4.1. Funciones para ejecutar búsqueda de contenido en elasticsearch 
Crear formato a partir de la respuesta de ES y convertir a DF:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": "",
    "esScore": 0
}

In [ ]:
from elasticsearch import helpers
from itertools import islice

def first_n(iterable, n):
    return islice(iterable, 0, n)

def parse_to_dataframe(es_candidates):
    from collections import defaultdict
    import pandas as pd
    results = defaultdict(list)
    for c in es_candidates:
        results['bookTitle'].append(c['_source']['bookTitle'])
        results['paragraphText'].append(c['_source']['paragraphText'])
        results['esScore'].append(c['_score'])
        results['paragraphId'].append(c['_source']['paragraphId'])
        results['bookURL'].append(c['_source']['bookURL'])
        results['bookId'].append(c['_source']['bookId'])
    return pd.DataFrame.from_dict(results)

def find_by_query(query, index_name="wikibooks-search-index", size=100):
    results = helpers.scan(es, query=query, index=index_name, preserve_order=True)
    if size:
        results = first_n(results, size)
    return parse_to_dataframe(list(results))

In [ ]:
# opción para NO truncar texto cuando es muy largo
pd.set_option("display.max_colwidth", None)

#### 4.2 Demo diferentes tipos de queries soportados por Elasticsearch

###### 4.2.1 Query usando un simple campo de búsqueda - paragraphText

In [ ]:
def es_query_1(user_query):
    es_query = {
        "query": {
            "bool": {
                "must": {"match": {"paragraphText": user_query}}
            }
        }
    }
    return es_query

###### 4.2.2 Query usando múltiples campos de búsqueda - paragraphText y bookTitle

In [ ]:
def es_query_2(user_query):
    es_query = {
        "query": {
            "multi_match": {
                "query": user_query,
                "fields": ["paragraphText", "bookTitle"],
                "tie_breaker": 1.0
            }
        }
    }
    return es_query

###### 4.2.3 Query impulsando campos de búsqueda paragraphText y bookTitle

In [ ]:
def es_query_3(user_query):
    es_query = {
        "query": {
            "multi_match": {
                "query": user_query,
                "fields": ["paragraphText", "bookTitle^2.0"],
                "tie_breaker": 1.0
            }
        }
    }
    return es_query

###### 4.2.4 Query usando múltiples campos de búsqueda con soporte a coincidencia exacta de frases

In [ ]:
import re
def es_query_4(user_query):
    
    match_queries = [
        {"match": {"paragraphText": user_query}},
        {"match": {"bookTitle": user_query}}
    ]
    
    # identificar frases en mi user_query
    phrases = re.findall(r'"([^"]+)"', user_query)
    # agregar frases a mi ES query
    for phrase in phrases:
        match_queries.append({"match": {"paragraphText": phrase}})
        match_queries.append({"match": {"bookTitle": phrase}})
    
    es_query = {
        "query": {
            "bool": {
                "should": match_queries
            }
        }
    }
    return es_query

### 5. Midiendo el desempeño de mi motor de búsqueda

##### 5.1 datos preparados para medición de desempeño

In [ ]:
query_relevant_answer_pairs = [
    {
        "text": "extensión total de la Antártida",
        "relevant_results": {
            "2803-1": "Su extensión total es de aproximadamente 14,2 millones de km2 en verano. Durante el invierno, la Antártida dobla su tamaño a causa de la gran cantidad de hielo marino que se forma en su periferia. El verdadero límite de la Antártida no es el litoral del continente en sí mismo, sino la Convergencia Antártica , que es una zona claramente definida en el extremo sur de los océanos Atlántico, Índico y Pacífico, entre los 48° y los 60° latitud S. En este punto, las corrientes frías que fluyen hacia el Norte desde la Antártida se mezclan con corrientes más cálidas en dirección Sur. La Convergencia Antártica marca una clara diferencia física en los océanos. Por estas razones el agua que rodea al continente antártico se considera un océano en sí mismo, a menudo llamado océano Glacial Antártico o Meridional."
        }
    },
    {
        "text": "dorsales submarinas",
        "relevant_results": {
            "3288-15": "Las dorsales son cordilleras submarinas formadas al ponerse en contacto el magma del interior de la Tierra con las aguas de los océanos. Por el centro del Atlántico corre una dorsal con forma de S, de unos 15 000 km de longitud, casi paralela a los continentes que da lugar a las islas Azores. El océano Índico también está recorrido por una dorsal de Norte a Sur y en el Pacífico también existen dorsales en dirección Noreste-Sureste; las islas Hawai están situadas sobre una de estas dorsales ."
        }
    },
    {
        "text": "principales gases en la atmósfera terrestre",
        "relevant_results": {
            "527-12": "La atmósfera terrestre es la parte gaseosa de la Tierra, siendo por esto la capa más externa. Está constituida por varios gases que varían en cantidad. Esta mezcla de gases que forma la atmósfera recibe genéricamente el nombre de aire. Los principales elementos que la componen son el oxígeno (21%) y el nitrógeno (78%)."
        }
    },
    {
        "text": "anatomía de los mayas",
        "relevant_results": {
            "318-10": "Eran de baja estatura, hombros anchos, pecho robusto, piernas cortas y musculosas, el rostro alargado y los pómulos salientes. Su estatura era, en término medio, de 1,65 m los varones y 1,42 m las mujeres. El cabello negro y lacio y la piel de color cobrizo confirma su procedencia del norte de Asia."
        }
    },
    {
        "text": "potencia máxima rueda hidráulica",
        "relevant_results": {
            "1965-3": "Los antiguos aprovechaban ya la energía del agua; utilizaban ruedas hidráulicas para moler trigo. Durante la Edad Media , las enormes ruedas hidráulicas de madera desarrollaban una potencia máxima de cincuenta caballos."
        }
    },
    {
        "text": "¿Cuándo se firmó el tratado antártico?",
        "relevant_results": {
            "2803-5": "Siete países ( Argentina, Australia, Chile, Francia, Gran Bretaña, Nueva Zelanda y Noruega ) reivindican la soberanía de ciertos territorios de la Antártida, pero desde el Tratado Antártico de 1961 estas demandas han sido abandonadas en favor de la cooperación internacional en las investigaciones científicas."
        }
    },
    {
        "text": "¿Cuánta sal hay en promedio en los océanos?",
        "relevant_results": {
            "3288-4": "El agua de los océanos contiene una media de 36 gramos por litro de sales. Esto se expresa diciendo que la salinidad media de los océanos es del 36 por 1 000. Entre las sales disueltas, el cloruro de sodio (sal común) es la más abundante. En las costas de las lugares donde hace mucho calor y la evaporación es grande el hombre extrae la sal de las salinas, por ejemplo en el Mediterráneo. también tiene disueltas pequeñas cantidades de yodo, fósforo y cobre."
        }
    },
    {
        "text": "¿Cuántos años tiene la tierra?",
        "relevant_results": {
            "2375-0": "Hace alrededor de 4550 millones de años atrás se formaron la Tierra y los otros planetas del Sistema Solar a partir de la nebulosa solar; una masa en forma de disco compuesta del polvo y gas que aún quedaba de la formación del Sol. Este proceso de formación de la Tierra tuvo lugar en un plazo de 10-20 millones de años."
        }
    },
    {
        "text": "¿Dónde vivía la abuelita de Caperucita?",
        "relevant_results": {
            "3152-2": "Caperucita tenía una abuela que vivía en una casita al otro lado del bosque, por lo que para ir a verla tenía que cruzar todo el bosque. En el bosque,vivían animales que no eran peligrosos: como los ciervos, los conejos y muchas especies de pájaros, pero también vivía en ese bosque un animal que sí podía ser peligroso, sobre todo para los niños: era un lobo que cuando tenía mucha hambre atacaba a las personas."
        }
    },
    {
        "text": "¿De qué estaba hecha la casa de Hansel y Gretel?",
        "relevant_results": {
            "2997-2": "Llegaron a una casita hecha de pan de jengibre, pastel y azúcar moreno,",
            "2997-4": "Después de dos días perdidos en el bosque, cuando ya no sabían más que hacer, los niños se detienen a escuchar el canto de un pájaro blanco al cual luego siguen hasta llegar a una casita hecha de pan de jengibre, pastel y azúcar morena. Hansel y Gretel empezaron a comer, pero lo que no sabían era que esta casita era la trampa de una vieja bruja para encerrarlos y luego comérselos."
        }
    }
]

##### 5.2 Funciones para medición de desempeño

In [ ]:
def recall_at_k(relevant_results, candidates, k):
    candidates_in_k = candidates[:min(k, len(candidates))]
    tp = len(set(relevant_results).intersection(set(candidates_in_k)))
    fn = len(set(relevant_results) - set(candidates_in_k))
    return tp / min(k, (tp + fn)) # same as tp / min(k, len(relevant_results))

def precision_at_k(relevant_results, candidates, k):
    candidates_in_k = candidates[:min(k, len(candidates))]
    tp = len(set(relevant_results).intersection(set(candidates_in_k)))
    fp = len(set(candidates_in_k) - set(relevant_results))
    return tp / min(k, (tp + fp)) # same as tp / min(k, len(candidates_in_k))

def average_precision_at_k(relevant_results, candidates, k):
    precision_acum = 0
    for i in range(k):
        if candidates[i] in relevant_results:
            rank = i + 1
            precision_acum += precision_at_k(relevant_results, candidates, rank)
    return precision_acum / min(k, len(relevant_results))

def compute_engine_performance(queries_with_relevant_answers, k_list, query_builder_functions):
    """
    Iterar sobre:
    - todos los tipos de query de Elasticsearch
    - k's = numbero de resultados tope que vamos a cubrir (top 10, 20, etc)
    - todos mis resultados relevantes previamente anotados
    
    Crear un DataFrame con la siguiente información:
    - tipo de query
    - K
    - recall
    - average_precision
    """
    import pandas as pd
    from collections import defaultdict
    performance_detailed_data = defaultdict(list)
    for query_builder in query_builder_functions:
        for k in k_list:
            for query_answers_pair in queries_with_relevant_answers:
                candidates_df = find_by_query(query_builder(query_answers_pair['text']), size=k)
                performance_detailed_data['query_builder'].append(query_builder.__name__)
                performance_detailed_data['k'].append(k)
                performance_detailed_data['query'].append(query_answers_pair['text'])
                if not candidates_df.empty:
                    ranked_candidates = candidates_df['paragraphId'].to_list()
                    relevant_results = list(query_answers_pair['relevant_results'].keys())
                    # metrics calculation
                    performance_detailed_data['recall'].append(recall_at_k(relevant_results, ranked_candidates, k))
                    performance_detailed_data['average_precision'].append(average_precision_at_k(relevant_results, ranked_candidates, k))
                else:
                    # metrics calculation
                    performance_detailed_data['recall'].append(0)
                    performance_detailed_data['average_precision'].append(0)
    performance_detailed = pd.DataFrame.from_dict(performance_detailed_data)
    performance_summary = performance_detailed[['query_builder', 'k', 'recall', 'average_precision']]
    performance_summary = performance_summary.groupby(['query_builder', 'k'], as_index=False).agg({"recall": "mean", "average_precision": "mean"})
    return performance_summary, performance_detailed

##### 5.3 Medición de desempeño de diferentes Elasticsearch queries

In [ ]:
performance_summary_es, _ = compute_engine_performance(query_relevant_answer_pairs, [10, 20], [es_query_1, es_query_2, es_query_3, es_query_4])
performance_summary_es

### 6 Creamos nuestra función de búsqueda final
Formato de respuesta esperado:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": "",
    "esScore": 0,
    "questionText": ""
}

In [ ]:
def format_es_response(user_question, es_candidates):
    results = list()
    for c in es_candidates:
        par = dict()
        par['questionText'] = user_question
        par['bookTitle'] = c['_source']['bookTitle']
        par['paragraphText'] = c['_source']['paragraphText']
        par['esScore'] = c['_score']
        par['paragraphId'] = c['_source']['paragraphId']
        par['bookURL'] = c['_source']['bookURL']
        par['bookId'] = c['_source']['bookId']
        results.append(par)
    return results

def search_candidates(user_question, index_name="wikibooks-search-index", size=20, es=Elasticsearch()):
    match_queries = [
        {"match": {"bookTitle": user_question}},
        {"match": {"paragraphText": user_question}}
    ]
    phrases = re.findall('"([^"]*)"', user_question)
    for phrase in phrases:
        match_queries.append({"match_phrase": {"bookTitle": phrase}})
        match_queries.append({"match_phrase": {"paragraphText": phrase}})

    es_query = {
        "query": {
            "bool": {
                "should": match_queries
            }
        }
    }

    results = helpers.scan(es, query=es_query, index=index_name, preserve_order=True)
    results = first_n(results, size)
    return format_es_response(user_question, results)
    

### 7 BONUS: Integrar modelos de reordenamiento

##### 7.1 Funciones para llamado de modelos de reordenamiento (reranking)

In [ ]:
# inicializamoss los modelos q utilizar
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Sequence Classification Model
tokenizer = AutoTokenizer.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")

# Extractive Question Answering Model - for Snippets
nlp = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',  
        {"use_fast": False}
    )
)

In [ ]:
# funciones utilizadas para normalizar, codificar, y rankear los parrafos usando the Sequence classification model
import os
from tqdm import tqdm
from joblib import Parallel, delayed

def normalize_pairs(pairs): # constants for default input normalization
    import re
    
    QUOTES_TRANSL_TABLE = {ord(x): ord(y) for x, y in zip(u"‘’´“”", u"'''\"\"")}
    
    def _fix_quotes(text):
        return text.translate(QUOTES_TRANSL_TABLE)
    def _normalize_query(query):
        return _fix_quotes(" ".join(query.split()).lower().replace("¿", "").replace("?", ""))
    def _normalize_paragraph(paragraph):
        return _fix_quotes(" ".join(paragraph.split()))
    
    return [(_normalize_query(row['questionText']),
             _normalize_paragraph(row['paragraphText'])) for row in pairs]
        
def encode_pairs(pairs_to_encode):
    return tokenizer.batch_encode_plus(
                normalize_pairs(pairs_to_encode),
                max_length=512,
                truncation="longest_first",
                return_tensors='pt',
                padding="longest"
            )

CPU_BATCH_SIZE = 5
def rank_paragraphs(pairs):
    pairs.sort(key=lambda x: (len(x["questionText"]) + len(x["paragraphText"])))
    result_tensors = list()
    for batch in tqdm([pairs[x:x + CPU_BATCH_SIZE] for x in range(0, len(pairs), CPU_BATCH_SIZE)]):
        encoded_pairs = encode_pairs(batch)
        # use Model to assign sequence (question -> passage) classification score
        predicted = model(**encoded_pairs)[0].softmax(dim=1)[:, 1]
        result_tensors.append((batch, predicted.to(device="cpu", non_blocking=True)))
    
    output = list()
    for batch, predicted in result_tensors:
        extracted_info = predicted.tolist()
        for index, score in enumerate(extracted_info):
            output.append({
                "questionText": batch[index]["questionText"],
                "bookId": batch[index]["bookId"],
                "bookTitle": batch[index]["bookTitle"],
                "bookURL": batch[index]["bookURL"],
                "paragraphId": batch[index]["paragraphId"],
                "paragraphText": batch[index]["paragraphText"],
                "sequenceScore": score
            })
    ranked_paragraphs = sorted(output, key=lambda item: item['sequenceScore'], reverse=True)
    return ranked_paragraphs
    
def get_answers_with_snippets(ranked_paragraphs, snippet_threshold=0.8):
    # get top parragraph for each document
    top_paragraph_by_book = dict()
    for par in ranked_paragraphs:
        if par['bookId'] in top_paragraph_by_book:
            continue
        top_paragraph_by_book[par['bookId']] = par
    top_answer_paragraphs = list(top_paragraph_by_book.values())
    # assign snippets
    for par in tqdm(top_answer_paragraphs):
        # use Model to extract answer from paragraph and assign the answer score
        if par['sequenceScore'] > snippet_threshold:
            extracted_answer = nlp(question=par['questionText'], context=par['paragraphText'])
            # print(f"{par['questionText']} - {extracted_answer['answer']} - {par['sequenceScore']}:{extracted_answer['score']}")
            if extracted_answer['score'] >= snippet_threshold:
                par['answerScore'] = extracted_answer['score']
                par['answerStart'] = extracted_answer['start']
                par['answerEnd'] = extracted_answer['end']
                par['answerText'] = extracted_answer['answer']
    return top_answer_paragraphs

##### 7.2 Funciones para medición de desempeño con reordenamiento

In [ ]:
def compute_engine_performance_reranker(queries_with_relevant_answers, k_list, best_es_query_builder, best_es_k):
    import pandas as pd
    from collections import defaultdict
    performance_detailed_data = defaultdict(list)
    for query_answers_pair in queries_with_relevant_answers:
        candidates_df = find_by_query(best_es_query_builder(query_answers_pair['text']), size=best_es_k)
        candidates_df['questionText'] = query_answers_pair['text']
        candidates_to_rank = candidates_df.to_dict("records")
        relevant_results = list(query_answers_pair['relevant_results'].keys())
        ranked_pairs = rank_paragraphs(candidates_to_rank)
        ranked_candidates = [p['paragraphId'] for p in ranked_pairs]
        for k in k_list:
            performance_detailed_data['k'].append(k)
            performance_detailed_data['query'].append(query_answers_pair['text'])
            # metrics calculation
            performance_detailed_data['recall'].append(recall_at_k(relevant_results, ranked_candidates, k))
            performance_detailed_data['average_precision'].append(average_precision_at_k(relevant_results, ranked_candidates, k))    
    performance_detailed = pd.DataFrame.from_dict(performance_detailed_data)
    performance_summary = performance_detailed[['k', 'recall', 'average_precision']]
    performance_summary = performance_summary.groupby('k', as_index=False).agg({"recall": "mean", "average_precision": "mean"})
    return performance_summary, performance_detailed

##### 7.3 Medición de desempeño de la función de reordenamiento (reranking)

In [ ]:
performance_summary_reranker, performance_detailed_reranker = compute_engine_performance_reranker(query_relevant_answer_pairs, [1, 5, 10, 15, 20],  es_query_4, 20)
performance_summary_reranker